In [ ]:
# All installations and checks
!export CUDA_VISIBLE_DEVICES=0,1,2,3
# !pip3 install transformers

In [ ]:
# All imports
import argparse
import os
import sys
import shutil
import datetime
import torch.utils.data as torch_data
from util import *
from featurizer import *
from base_model import *
from torch_model import *
from evaluator import *

In [ ]:
# Global initializations and functions


In [ ]:
# Create training data (i/o)

config = read_conf("wikisql_demo.conf")

featurizer = HydraFeaturizer(config)
train_data = SQLDataset(config["train_data_path"], config, featurizer, True)
train_data_loader = torch_data.DataLoader(train_data, batch_size=int(config["batch_size"]), shuffle=True, pin_memory=True)

num_samples = len(train_data)

In [ ]:
# Creating the model and showing number of batches to be processed
note = ""
model_path = "model--"+datetime.datetime.now().strftime("%Y%m%d_%H%M%S")
config["num_train_steps"] = int(num_samples * int(config["epochs"]) / int(config["batch_size"]))
step_per_epoch = num_samples / int(config["batch_size"])
print("total_steps: {0}, warm_up_steps: {1}".format(config["num_train_steps"], config["num_warmup_steps"]))

model = create_model(config, is_train=True)
evaluator = HydraEvaluator(model_path, config, featurizer, model, note)


In [ ]:
# Starting training
print("start training")
print(model_path)
loss_avg, step, epoch = 0.0, 0, 0
while True:
    for batch_id, batch in enumerate(train_data_loader):
        print(batch_id)
        cur_loss = model.train_on_batch(batch)
        loss_avg = (loss_avg * step + cur_loss) / (step + 1)
        step += 1
        print("here1")
        if batch_id % 1 == 0:
            print("here2")
            currentDT = datetime.datetime.now()
            print("[{3}] epoch {0}, batch {1}, batch_loss={2:.4f}".format(epoch, batch_id, cur_loss,
                                                                            currentDT.strftime("%m-%d %H:%M:%S")))
    model.save(model_path, epoch)
    evaluator.eval(epoch)
    epoch += 1
    if epoch >= int(config["epochs"]):
        break


In [ ]:
# Predicting SQL output of a single question input
config_demo = read_conf("wikisql_demo.conf")
featurizer_demo = HydraFeaturizer(config_demo)
model_path_demo = "model--20221123_123520"
epoch_demo = 0
model_demo = create_model(config_demo, is_train=True)
note = "_demo"
evaluator_demo = HydraEvaluator(model_path_demo, config_demo, featurizer_demo, model_demo, note)
model_demo.load(model_path_demo, epoch_demo)
evaluator_demo.eval(epoch_demo)